# Library

In [2]:
import tensorflow  as tf
import pandas  as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
import random

In [3]:
from textblob import TextBlob
from nltk.tokenize.toktok import ToktokTokenizer
import re
tokenizer=ToktokTokenizer()
import spacy
from string import punctuation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity
import en_core_web_sm
import torch



import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import spacy

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
spc = spacy.load("en_core_web_sm")

nlp=spacy.load('en_core_web_sm',disable=['ner'])
spc_en = en_core_web_sm.load()


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tazkiiia\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Tazkiiia\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Tazkiiia\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Content Based using TF-IDF

In [18]:
df = pd.read_csv(r'D:\Bismillah SKRIPSIIIII\REVISI\dataset\data_sampled.csv')
df.head()

,name,course_url,course_id,reviews,reviewers,rating,user_id,Sequence
0,Machine Learning Foundations: A Case Study App...,https://www.coursera.org/learn/ml-foundations,1,informative basic machine learning set stage d...,Satish K D,5.0,60454,machine learning foundation case study approac...
1,Machine Learning Foundations: A Case Study App...,https://www.coursera.org/learn/ml-foundations,1,great learning experience option video exercis...,Mohit A,4.0,7170,machine learning foundation case study approac...
2,Machine Learning Foundations: A Case Study App...,https://www.coursera.org/learn/ml-foundations,1,love course hope soon,Hari S,5.0,64355,machine learning foundation case study approac...
3,Machine Learning Foundations: A Case Study App...,https://www.coursera.org/learn/ml-foundations,1,easy follow enjoyable,Mai T,5.0,130375,machine learning foundation case study approac...
4,Machine Learning Foundations: A Case Study App...,https://www.coursera.org/learn/ml-foundations,1,really like course nice introduction ml downsi...,Ornella G,4.0,50513,machine learning foundation case study approac...


In [19]:
import string

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Fungsi membersihkan teks
def clean_text(text):
    if isinstance(text, str):
        text = re.sub(r'_+', ' ', text)
        text = re.sub(r'[^a-zA-Z\s]', '', text)
        text = text.lower()
        text = ''.join([c for c in text if c not in string.punctuation])
        text = re.sub(r'\d+', '', text)
        text = re.sub(' +', ' ', text).strip()
        text = ' '.join([lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words])
        return text
    return ""


# Fit TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Sequence'])


In [20]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix

# Fit TF-IDF Vectorizer pada kolom Sequence
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Sequence'])

# Simpan matriks TF-IDF dalam format sparse
df['tfidf_matrix'] = list(csr_matrix(tfidf_matrix))

# Tampilkan hasil
df.head()


,name,course_url,course_id,reviews,reviewers,rating,user_id,Sequence,tfidf_matrix
0,Machine Learning Foundations: A Case Study App...,https://www.coursera.org/learn/ml-foundations,1,informative basic machine learning set stage d...,Satish K D,5.0,60454,machine learning foundation case study approac...,"(0, 1057)\t0.2349728003398067\n (0, 492)\t0..."
1,Machine Learning Foundations: A Case Study App...,https://www.coursera.org/learn/ml-foundations,1,great learning experience option video exercis...,Mohit A,4.0,7170,machine learning foundation case study approac...,"(0, 2681)\t0.40035871121938726\n (0, 2183)\..."
2,Machine Learning Foundations: A Case Study App...,https://www.coursera.org/learn/ml-foundations,1,love course hope soon,Hari S,5.0,64355,machine learning foundation case study approac...,"(0, 2827)\t0.4490083042923362\n (0, 1463)\t..."
3,Machine Learning Foundations: A Case Study App...,https://www.coursera.org/learn/ml-foundations,1,easy follow enjoyable,Mai T,5.0,130375,machine learning foundation case study approac...,"(0, 982)\t0.481110595329629\n (0, 1244)\t0...."
4,Machine Learning Foundations: A Case Study App...,https://www.coursera.org/learn/ml-foundations,1,really like course nice introduction ml downsi...,Ornella G,4.0,50513,machine learning foundation case study approac...,"(0, 1360)\t0.3356856830521129\n (0, 3410)\t..."


In [21]:
df.to_csv('xxx.csv', index=False)

In [9]:
def recommend_courses(user_input, tfidf_matrix, df, top_n=10):
    user_input_processed = clean_text(user_input) 
    user_tfidf = tfidf_vectorizer.transform([user_input_processed])
    cosine_similarities = cosine_similarity(user_tfidf, tfidf_matrix).flatten()
    
    # Get top N recommendations
    recommended_indices = cosine_similarities.argsort()[::-1]  
    recommendations = df.iloc[recommended_indices][['course_id', 'name', 'course_url', 'rating']]
    recommendations['similarity_score'] = cosine_similarities[recommended_indices]
    
    
    unique_recommendations = recommendations.drop_duplicates(subset='course_id').head(top_n)
    return unique_recommendations


user_input = "I want to learn about data science for beginner"
recommendations = recommend_courses(user_input, tfidf_matrix, df)
recommendations


,course_id,name,course_url,rating,similarity_score
187,11,What is Data Science?,https://www.coursera.org/learn/what-is-datasci...,5.0,0.654016
354,20,Python for Data Science and AI,https://www.coursera.org/learn/python-for-appl...,5.0,0.493167
137,8,Introduction to Data Science in Python,https://www.coursera.org/learn/python-data-ana...,5.0,0.462768
101,6,Python Data Structures,https://www.coursera.org/learn/python-data,5.0,0.376668
1283,72,Databases and SQL for Data Science,https://www.coursera.org/learn/sql-data-science,4.0,0.362269
507,29,Machine Learning for All,https://www.coursera.org/learn/uol-machine-lea...,4.0,0.326058
494,28,SQL for Data Science,https://www.coursera.org/learn/sql-for-data-sc...,5.0,0.316759
1948,45,Python Basics,https://www.coursera.org/learn/python-basics,4.0,0.315171
840,47,Data Science Methodology,https://www.coursera.org/learn/data-science-me...,5.0,0.313315
1885,105,Introduction to Probability and Data with R,https://www.coursera.org/learn/probability-intro,5.0,0.309903


# Content Based using BERT

In [10]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
import re
import string

c:\Users\Tazkiiia\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
# Load dataset
df = pd.read_csv(r'D:\Bismillah SKRIPSIIIII\REVISI\model\xxx.csv')
df.head()

,name,course_url,course_id,reviews,reviewers,rating,user_id,Sequence,TF-IDF_Vector
0,Machine Learning Foundations: A Case Study App...,https://www.coursera.org/learn/ml-foundations,1,informative basic machine learning set stage d...,Satish K D,5.0,60454,machine learning foundation case study approac...,"(0, 1057)\t0.2349728003398067\n (0, 492)\t0..."
1,Machine Learning Foundations: A Case Study App...,https://www.coursera.org/learn/ml-foundations,1,great learning experience option video exercis...,Mohit A,4.0,7170,machine learning foundation case study approac...,"(0, 2681)\t0.40035871121938726\n (0, 2183)\..."
2,Machine Learning Foundations: A Case Study App...,https://www.coursera.org/learn/ml-foundations,1,love course hope soon,Hari S,5.0,64355,machine learning foundation case study approac...,"(0, 2827)\t0.4490083042923362\n (0, 1463)\t..."
3,Machine Learning Foundations: A Case Study App...,https://www.coursera.org/learn/ml-foundations,1,easy follow enjoyable,Mai T,5.0,130375,machine learning foundation case study approac...,"(0, 982)\t0.481110595329629\n (0, 1244)\t0...."
4,Machine Learning Foundations: A Case Study App...,https://www.coursera.org/learn/ml-foundations,1,really like course nice introduction ml downsi...,Ornella G,4.0,50513,machine learning foundation case study approac...,"(0, 1360)\t0.3356856830521129\n (0, 3410)\t..."


In [12]:
# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')


In [13]:
# Function to clean text
def clean_text(text):
    if isinstance(text, str):
        text = re.sub(r'_+', ' ', text)
        text = re.sub(r'[^a-zA-Z\s]', '', text)
        text = text.lower()
        text = re.sub(' +', ' ', text).strip()
        return text
    return ""

# Preprocess text
# df['Sequence'] = df['Sequence'].apply(clean_text)


In [16]:
# Function to generate BERT embeddings
def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', max_length=512, truncation=True, padding='max_length')
    with torch.no_grad():
        outputs = model(**inputs)
    # Use the [CLS] token's embedding
    cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
    return cls_embedding

# print("Generating BERT embeddings for dataset...")
# df['bert_embedding'] = df['Sequence'].apply(get_bert_embedding)


In [ ]:
# df.to_csv('final_df_bert.csv', index=False)

In [ ]:
# Function to recommend courses using BERT embeddings
def recommend_courses_bert(user_input, df, top_n=10):
    user_input_cleaned = clean_text(user_input)
    user_input_embedding = get_bert_embedding(user_input_cleaned)
    
    # Compute cosine similarity between user input and all courses
    similarities = []
    for embedding in df['bert_embedding']:
        similarity = cosine_similarity([user_input_embedding], [embedding])[0][0]
        similarities.append(similarity)
    
    # Add similarities to the dataframe
    df['similarity_score'] = similarities
    
    # Get top N recommendations
    recommendations = df.sort_values(by='similarity_score', ascending=False)
    unique_recommendations = recommendations.drop_duplicates(subset='course_id').head(top_n)
    return unique_recommendations[['course_id', 'name', 'course_url', 'rating', 'similarity_score']]

# Example usage
user_input = "I want to learn about data science for beginner"
recommendations = recommend_courses_bert(user_input, df)
print(recommendations)
